In [0]:
%scala

import com.tomtom.orbis.io.spark.model.OrbisElement;
import org.apache.spark.api.java.function.FilterFunction;
import org.apache.spark.sql.functions
import com.tomtom.addressing.bulk.commons.config.SourceConfigLoader
import org.apache.sedona.spark.SedonaContext
import org.apache.sedona.viz.core.Serde.SedonaVizKryoRegistrator

val session = SedonaContext.builder()
            .config("spark.kryo.registrator", classOf[SedonaVizKryoRegistrator].getName)
            .appName("dbscanOnAptSnapshot")
            .getOrCreate()
val spark = SedonaContext.create(session)
implicit val sparky = spark

val aptDataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.checks_input_features
""").as("apt").withColumn("apt_geom", functions.expr("ST_GeomFromText(apt.location)"));

val aerodromeDataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.layer_19174
""").withColumn("visualization_geom", functions.expr("ST_GeomFromText(wkt)"))

val bfpDataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.layer_53801
""")


val visualizationOfInterestDataset = spark.sql("""
    SELECT *
    FROM 
    preprocess_dev.layer_10698
""")

println(s"aptDataset:${aptDataset.count}")
println(s"aerodromeDataset:${aerodromeDataset.count}")
println(s"bfpDataset:${bfpDataset.count}")
println(s"visualizationOfInterestDataset:${visualizationOfInterestDataset.count}")

In [0]:
%scala
import org.apache.spark.sql.functions
import org.apache.spark.sql.functions.when
import org.apache.spark.sql.functions.col
import com.tomtom.addressing.bulk.commons.config.SourceConfigLoader
import org.apache.sedona.spark.SedonaContext
import org.apache.sedona.viz.core.Serde.SedonaVizKryoRegistrator

val session = SedonaContext.builder()
            .config("spark.kryo.registrator", classOf[SedonaVizKryoRegistrator].getName)
            .appName("dbscanOnAptSnapshot")
            .getOrCreate()
val spark = SedonaContext.create(session)
implicit val sparky = spark

val APT_ON_RUNWAY = "apt_on_runway";
val APT_ON_AERODROME = "apt_on_aerodrome";
val APT_ON_BFP = "apt_on_bfp_inside_aerodrome";
val APT_ON_VIS_OF_INTEREST = "apt_on_vis_of_interest";
val APT_VISUALIZATION_ID = "apt.visualization_id";
val APT_APT_GEOM = "apt.apt_geom";
val APT_FEATURE_ID = "apt.featureId";
val FEATURE_ID = "featureId";
val APT_GEOM = "apt_geom";
val VISUALIZATION_ID = "visualization_id";
val VISUALIZATION_GEOM = "visualization_geom";
val RUNWAY_ID = "runway_id";

val aptOnAerodromeDataset = aptDataset.as("apt")
                .join(aerodromeDataset.as("visualization"), functions.expr("ST_Intersects(visualization.visualization_geom, apt.apt_geom)"), "inner")
                .withColumn(APT_ON_AERODROME, when(col("visualization.visualization_geom") === null, false).otherwise(true))
                // .select(
                //         col(APT_FEATURE_ID).as(FEATURE_ID),
                //         col(APT_APT_GEOM).as(APT_GEOM),
                //         col("visualization.id").as(VISUALIZATION_ID),
                //         col("visualization.visualization_geom").as(VISUALIZATION_GEOM),
                //         col(APT_ON_AERODROME).as(APT_ON_AERODROME)
                // );

display(aptOnAerodromeDataset)

In [0]:
%scala
println(s"aptOnAerodromeDataset count:${aptOnAerodromeDataset.count}")